In [1]:
import cv2
import numpy as np
from pynput.keyboard import Controller,Key
from playsound import playsound

keyboard=Controller()

   
cap_path="Videos\\video8.mp4"
Yolo_v4_model_path="Models\\yolov4_road_model\\yolov4_road_v2_final.weights"
Yolo_v4_cfg_path="Models\\yolov4_road_model\\yolov4_road.cfg"
color=(255,0,0)
view_color=(1,1,1)
labels=["road"]

print("[INFO] Road Model Loading...")
model=cv2.dnn.readNetFromDarknet(Yolo_v4_cfg_path, Yolo_v4_model_path)
print("[INFO] Road Model Loaded...")

layers=model.getLayerNames()
for layer in model.getUnconnectedOutLayers():
    output_layer=[layers[layer-1]]
    

Person_Yolo_v4_model_path="Models\\yolov4_three_class_model\\yolov4_v2_final.weights"
Person_Yolo_v4_cfg_path="Models\\yolov4_three_class_model\\yolov4_v2.cfg"
person_color=(0,255,0)
person_labels=["person","face","hand"]

print("[INFO] Person Model Loading...")
person_model=cv2.dnn.readNetFromDarknet(Person_Yolo_v4_cfg_path, Person_Yolo_v4_model_path)
print("[INFO] Person Model Loaded...")

person_layers=person_model.getLayerNames()
for layer in person_model.getUnconnectedOutLayers():
    person_output_layer=[layers[layer-1]]
    
            
cap=cv2.VideoCapture(cap_path)

ret,frame=cap.read()
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
imageOrj=frame[0:height,int(0+(width/4)):int(width-(width/4)),:]
bboxArea=imageOrj.shape[0]*imageOrj.shape[1]
t_=2
time=t_
refresh_time=0
while True:
    ret,frame=cap.read()
    if ret==True:
        image=frame[0:height,int(0+(width/4)):int(width-(width/4)),:]
        
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        img_blob=cv2.dnn.blobFromImage(frame, 1/255, (416,416), swapRB= True, crop= False)  
        
        
        
        model.setInput(img_blob)
        
        detection_layers=model.forward(output_layer)


        for detection_layer in detection_layers:
            for object_detection in detection_layer:

                scores=object_detection[5:]
                predicted_id=np.argmax(scores)
                confidence=scores[predicted_id]

                if confidence > 0.75:

                    label=labels[predicted_id]
                    bounding_box=object_detection[0:4]*np.array([width,height,width,height])
                    (box_center_x, box_center_y, box_width, box_height)= bounding_box.astype("int")

                    start_x=int(box_center_x-(box_width/2))
                    start_y=int(box_center_y-(box_height/2))

                    end_x=start_x+box_width
                    end_y=start_y+box_height

#                     print("{} {} {} {}".format(box_center_x, box_center_y, box_width, box_height))
#                     print("{} {} {} {}".format(start_x, start_y, end_x, end_y))

                    label="{}: {:.2f}%".format(label, confidence*100)
#                     print("predicted object {}".format(label))

                    cv2.rectangle(frame, (start_x, start_y),(end_x, end_y), color, 1) 
                    cv2.putText(frame, label, (start_x,start_y+10),cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)

        
        person_model.setInput(img_blob)
        
        person_detection_layers=person_model.forward(person_output_layer)


        for person_detection_layer in person_detection_layers:
            for person_object_detection in person_detection_layer:

                person_scores=person_object_detection[5:]
                person_predicted_id=np.argmax(person_scores)
                person_confidence=person_scores[person_predicted_id]

                if person_confidence > 0.80:

                    person_label=person_labels[person_predicted_id]
                    person_bounding_box=person_object_detection[0:4]*np.array([width,height,width,height])
                    (box_center_x, box_center_y, box_width, box_height)= person_bounding_box.astype("int")

                    person_start_x=int(box_center_x-(box_width/2))
                    person_start_y=int(box_center_y-(box_height/2))

                    person_end_x=person_start_x+box_width
                    person_end_y=person_start_y+box_height

#                     print("{} {} {} {}".format(box_center_x, box_center_y, box_width, box_height))
#                     print("{} {} {} {}".format(start_x, start_y, end_x, end_y))

                    person_label="{}: {:.2f}%".format(person_label, person_confidence*100)
#                     print("predicted object {}".format(label))

                    cv2.rectangle(frame, (person_start_x, person_start_y),(person_end_x, person_end_y), person_color, 1) 
                    cv2.putText(frame, person_label, (person_start_x,person_start_y+10),cv2.FONT_HERSHEY_SIMPLEX, 0.3, person_color, 1)

        
        

        diff=cv2.subtract(imageOrj,image)

        ret,thresh=cv2.threshold(diff[:,:,0],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        cnts=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]

        thresh=cv2.dilate(thresh,None,iterations=2)
        if not cnts:
            c=bboxArea
            ratio=0.0
            c_max=0.0
        else:
            c=max(cnts,key=cv2.contourArea)
            ratio=cv2.contourArea(c)/bboxArea
            c_max=cv2.contourArea(c)
        # if ratio>0.2 and ratio<0.3:
        #     keyboard.press(Key.space)
        # # if ratio>0.65:
        # #     keyboard.press(ord("q"))
        time-=1
        if time==0:
            ret,orj_frame=cap.read()
            imageOrj=orj_frame[0:height,int(0+(width/4)):int(width-(width/4)),:]
            bboxArea=imageOrj.shape[0]*imageOrj.shape[1]
            time=t_
            refresh_time+=1
            
        cv2.putText(frame,"ImageOrj Refreshed: "+str(refresh_time)+" times",(10,455),cv2.FONT_HERSHEY_SIMPLEX,0.3,(175,175,175),1)
   
        cv2.putText(frame,("TimeLeft: "+str(time)),(int(width-(width/4)+5),25),cv2.FONT_HERSHEY_SIMPLEX,0.3,(175,175,175),1)

        if ratio>0.10:
            playsound ("Voices\\radyoaktif.mp3")
            cv2.putText(frame,"Attention!!!",(int(width-(width/4)+5),240),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)



        # if time.sleep(5) is False:
        #     ret,frame=cam.read_frame()
        #     imageOrj=frame[0:480,160:480,:]
        #     bboxArea=imageOrj.shape[0]*imageOrj.shape[1]
        #     print("İmageOrj Refreshed")


        cv2.putText(diff,("ratio: "+str(ratio)),(10,10),cv2.FONT_HERSHEY_SIMPLEX,0.3,(175,175,175),1)
        cv2.imshow("diff",diff)
        cv2.imshow("ImageOrj",imageOrj)

        cv2.putText(thresh,str(c_max),(10,10),cv2.FONT_HERSHEY_SIMPLEX,0.3,(175,175,175),1)
        cv2.imshow("thresh",thresh)

        cv2.rectangle(frame,(int(0+(width/4)),0),(int(width-(width/4)),height),view_color,1)
        cv2.putText(frame,"@author: muhammetsarican",(10,25),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,0.4,(255,255,255),1)
#         cv2.imshow("Frame",frame)

            
            
            
            
        cv2.imshow('frame',frame)
        key=cv2.waitKey(1)&0xff

        if key == ord('q'):
            break
cap.release()        
cv2.destroyAllWindows()

[INFO] Model Loading...
[INFO] Model Loaded...
[INFO] Model Loading...
[INFO] Model Loaded...
